In [1]:
normal_label = "five"
normal_class = 5

# Generate Prompt

In [2]:
import gensim
import gensim.downloader
from nltk.stem import WordNetLemmatizer
import numpy as np
import re
from torchvision.datasets import MNIST

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
model = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
wl =WordNetLemmatizer()

In [6]:
pattern = '[a-z]+'
topn = 1000
all_similars = model.most_similar(positive=[normal_label], topn = topn)
normalized_similars_words = []
normalized_similars_probs = []
for similar in all_similars:
    normalized = wl.lemmatize(similar[0].lower())
    result = re.match(pattern, normalized)
    if result is None or result.group() != normalized:
        continue
    if normalized != normal_label and normalized not in normalized_similars_words:
        normalized_similars_words.append(normalized)
        normalized_similars_probs.append(similar[1] ** 10)    

probabilities = np.zeros(len(normalized_similars_probs) + 1)
probabilities[1:] = np.array(normalized_similars_probs) / sum(normalized_similars_probs)

cum_prob = np.cumsum(probabilities, axis=-1)[:, None]
r = np.random.uniform(size=(50))
samples = np.argmax((cum_prob > r).T, axis=-1)
prompts = []
for i in samples:
    prompts.append(normalized_similars_words[i-1])

In [7]:
negative_adjectives = [
    f'A photo of {normal_label} with a crack',
    f'A photo of a broken {normal_label}',
    f'A photo of {normal_label} with a defect', 
    f'A photo of {normal_label} with damage',
    f'A photo of {normal_label} with a scratch', 
    f'A photo of {normal_label} with a hole',
    f'A photo of {normal_label} torn',
    f'A photo of {normal_label} cut',
    f'A photo of {normal_label} with contamination',
    f'A photo of {normal_label} with a fracture',
    f'A photo of a damaged {normal_label}',
    f'A photo of a fractured {normal_label}',
    f'A photo of {normal_label} with destruction',
    f'A photo of {normal_label} with a mark',
]

In [8]:
prompts = prompts + negative_adjectives

In [9]:
prompts

['three',
 'eleven',
 'three',
 'six',
 'eight',
 'seven',
 'thirty',
 'eight',
 'seven',
 'two',
 'seven',
 'six',
 'four',
 'six',
 'four',
 'eight',
 'nine',
 'eight',
 'seven',
 'seven',
 'seven',
 'seven',
 'seven',
 'eight',
 'six',
 'three',
 'four',
 'nine',
 'sixteen',
 'three',
 'seven',
 'six',
 'nine',
 'three',
 'eight',
 'seven',
 'eight',
 'eight',
 'seven',
 'six',
 'four',
 'nine',
 'four',
 'six',
 'two',
 'six',
 'four',
 'three',
 'seven',
 'three',
 'A photo of five with a crack',
 'A photo of a broken five',
 'A photo of five with a defect',
 'A photo of five with damage',
 'A photo of five with a scratch',
 'A photo of five with a hole',
 'A photo of five torn',
 'A photo of five cut',
 'A photo of five with contamination',
 'A photo of five with a fracture',
 'A photo of a damaged five',
 'A photo of a fractured five',
 'A photo of five with destruction',
 'A photo of five with a mark']

# Generate Image

In [10]:
batch_size = 20

In [22]:
%%capture
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install gdown

import numpy as np

import gdown
!gdown --id 1cZwCFQOSUq4w2ckZjhFRlJmZ5cvSdgbs

import zipfile
with zipfile.ZipFile("./glide-text2im-code.zip", 'r') as zip_ref:
    zip_ref.extractall("./")


!pip install ftfy
import ftfy

import sys
#!git clone https://github.com/openai/glide-text2im.git
sys.path.append('./glide-text2im')



import torch
import matplotlib.pyplot as plt

In [23]:
def show_samples(x):
    x = x.permute(0, 2, 3, 1).detach().cpu().numpy()
    img = image_grid(x)
    plt.figure(figsize=(4,4))
    plt.axis('off')
    plt.imshow(img)
    plt.show()
def image_grid(x):
    size =64# config.data.image_size
    channels =3# config.data.num_channels
    img = x.reshape(-1, size, size, channels)
    w = int(np.sqrt(img.shape[0]))
    img = img.reshape((w, w, size, size, channels)).transpose((0, 2, 1, 3, 4)).reshape((w * size, w * size, channels))
    return img

In [24]:
from PIL import Image
from IPython.display import display
import torch as th
import torch.nn as nn

from glide_text2im.clip.model_creation import create_clip_model
from glide_text2im.download import load_checkpoint
from glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
    model_and_diffusion_defaults_upsampler,
)
from glide_text2im.tokenizer.simple_tokenizer import SimpleTokenizer

In [25]:
has_cuda = th.cuda.is_available()
device = th.device('cpu' if not has_cuda else 'cuda')

In [26]:
options = model_and_diffusion_defaults()
options['use_fp16'] = has_cuda
options['timestep_respacing'] = '100' # use 100 diffusion steps for fast sampling
model, diffusion = create_model_and_diffusion(**options)
model.eval()
if has_cuda:
    model.convert_to_fp16()
model.to(device)
model.load_state_dict(load_checkpoint('base', device))
print('total base parameters', sum(x.numel() for x in model.parameters()))

total base parameters 385030726


In [27]:
clip_model = create_clip_model(device=device)
clip_model.image_encoder.load_state_dict(load_checkpoint('clip/image-enc', device))
clip_model.text_encoder.load_state_dict(load_checkpoint('clip/text-enc', device))

<All keys matched successfully>

In [28]:
def show_images(batch: th.Tensor):
    """ Display a batch of images inline. """
    scaled = ((batch + 1)*127.5).round().clamp(0,255).to(th.uint8).cpu()
    reshaped = scaled.permute(2, 0, 3, 1).reshape([batch.shape[2], -1, 3])
    display(Image.fromarray(reshaped.numpy()))

In [29]:
guidance_scale = 10
upsample_temp = 0.997

In [30]:
img_transform_simple = transforms.Compose([transforms.Grayscale(3), transforms.ToTensor(),transforms.Resize((64,64))])
train_set = MNIST(root = './', train=True, download=True, transform=img_transform_simple)
idx = (torch.tensor(train_set.targets) == normal_class) 
train_set.targets = torch.tensor(train_set.targets)[idx]
train_set.data = train_set.data[idx]
images_train=[]
for x,_ in train_set:
    images_train.append(x)
DESIRED_NUMBER_OF_SAMPLES_GENERATED = len(images_train)
images_train_t = torch.stack(images_train)
train_loader = DataLoader(images_train_t, batch_size=batch_size, shuffle=True)

<ipython-input-30-854a825c6c85>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = (torch.tensor(train_set.targets) == normal_class)
<ipython-input-30-854a825c6c85>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_set.targets = torch.tensor(train_set.targets)[idx]


In [31]:
import torchvision.transforms as transforms
trans_to_32 = transforms.Compose([  transforms.Resize((32,32))])

In [ ]:
import random

all_generated_samples = []

last_save = 50
save_step = 50

def desired_no_samples_reached():
    return len(all_generated_samples) * batch_size >= DESIRED_NUMBER_OF_SAMPLES_GENERATED

while not desired_no_samples_reached():
    for x in train_loader: 
        x = x.to(device)
        random_name = random.choice(prompts)        
        print("prompt: ", random_name)
        tokens = model.tokenizer.encode(random_name)
        init = x.mul(2).sub(1)
        batch_size = x.shape[0]

        tokens, mask = model.tokenizer.padded_tokens_and_mask(
            tokens, options['text_ctx']
        )

        # Pack the tokens together into model kwargs. init batch_size
        model_kwargs = dict(
            tokens=th.tensor([tokens] * batch_size, device=device),
            mask=th.tensor([mask] * batch_size, dtype=th.bool, device=device),
        )

        # Setup guidance function for CLIP model.
        cond_fn = clip_model.cond_fn([random_name] * batch_size, guidance_scale)

        # Sample from the base model. batch
        model.del_cache()
        generated_samples = diffusion.p_sample_loop(
            model,
            (batch_size, 3, options["image_size"], options["image_size"]),
            device=device,
            clip_denoised=True,
            progress=True,
            model_kwargs=model_kwargs,
            cond_fn=cond_fn,
            skip_timesteps=5,
            init_image=init.cuda()
        )
        model.del_cache()
        all_generated_samples.append(generated_samples.detach().cpu())
        all_generated_samples_tensor = torch.cat(all_generated_samples)
        all_generated_samples_tensor = trans_to_32(all_generated_samples_tensor)
        
        # Show the output
        if all_generated_samples_tensor.size(0) >= last_save or desired_no_samples_reached():
            last_save += save_step
            with open(f'./MNIST_GLIDE_NormalClass_{normal_label}.npy', 'wb') as f:
                np.save(f, all_generated_samples_tensor.detach().cpu().numpy())
            print(f"checkpoint saved with {len(all_generated_samples) * batch_size} samples")
        show_images(init)
        show_images(generated_samples)
        if desired_no_samples_reached():
            break
        

prompt:  four


  0%|          | 0/95 [00:00<?, ?it/s]